In [11]:
!pip install -q transformers datasets accelerate scikit-learn pandas
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [13]:
df = pd.read_csv("FinancES_train_kaggle.csv", encoding="utf-8")
print("Dataset shape:", df.shape)
print(df.head())

# Separación en Train/Val: 80% / 20%
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)

Dataset shape: (6359, 3)
   id                                               text  label
0   0  Renfe afronta mañana un nuevo día de paros par...      2
1   1       Presupuesto populista con cimientos frágiles      2
2   2  Biden no cree que la OPEP+ vaya a ayudar con l...      2
3   3  La deuda de las familias cae en 25.000 millone...      0
4   4  Bestinver: no hay "momento más inoportuno" par...      2


In [14]:
model_checkpoint = "UMUTeam/roberta-spanish-sentiment-analysis"
num_labels = 3  # positivo(0), neutro(1), negativo(2)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset   = val_dataset.map(tokenize_function, batched=True)

# Eliminamos columnas que no se usan por el modelo
train_dataset = train_dataset.remove_columns(["id", "text"])
val_dataset   = val_dataset.remove_columns(["id", "text"])

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Map:   0%|          | 0/5087 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "f1_macro": f1_score(labels, preds, average="macro")
    }

In [18]:
training_args = TrainingArguments(
    output_dir="roberta-spanish-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Evaluación en validación
metrics = trainer.evaluate()
print("F1 Macro (validación):", metrics["eval_f1_macro"])

<ipython-input-19-d590b701c615>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.531000,0.527540,0.735335
2,0.286200,0.582576,0.740006
3,0.116400,0.853266,0.737112
4,0.032700,1.066623,0.737076
5,0.013400,1.118137,0.728793


F1 Macro (validación): 0.7400058678673941


In [21]:
test_df = pd.read_csv("FinancES_test_kaggle.csv", encoding="utf-8")

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.remove_columns(["id", "text"])

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

submission = pd.DataFrame({
    "id": test_df["id"],
    "label": preds
})

submission.to_csv("submission_roberta_spanish.csv", index=False)
print("Archivo 'submission_roberta_spanish.csv' creado con las predicciones.")

Map:   0%|          | 0/1621 [00:00<?, ? examples/s]

Archivo 'submission_roberta_spanish.csv' creado con las predicciones.
